In [ ]:
# Andy Vo
# ECGR 4106 Homework 2
# Due: 3/04/2022

In [ ]:
# Problem 1

# read datafile
%matplotlib inline
import numpy as np
import pandas as pd
import torch
import torch.optim as optim

data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ECGR_4106_Homework/Homework1/Text_Files/Housing.csv")
housing_df=pd.DataFrame(data)
x_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
y_vars = ['price']
x_df = housing_df[x_vars]
y_df = housing_df[y_vars]

torch.set_printoptions(edgeitems=2, linewidth=75)

In [ ]:
def model (params, x):
  return params[5]*x[:,4] + params[4]*x[:,3] + params[3]*x[:,2]
  + params[2]*x[:,1] + params[1]*x[:,0] + params[0]
def loss_fn (y_p, y):
  squared_diffs = (y_p -y)**2
  return squared_diffs.mean()
def dloss_fn(y_p, y):
  return 2*(y_p-y)/y_p.size(0)
def dmodel_dw4(w,x):
  return x[:,4]
def dmodel_dw3(w,x):
  return x[:,3]
def dmodel_dw2(w,x):
  return x[:,2]
def dmodel_dw1(w,x):
  return x[:,1]
def dmodel_dw0(w,x):
  return x[:,0]
def dmodel_db(params):
  return 1.0
def grad_fn(x,y,y_p,params):
  dloss_dyp = dloss_fn(y_p,y)
  dloss_dw4 = dloss_dyp * dmodel_dw4 (params,x)
  dloss_dw3 = dloss_dyp * dmodel_dw3 (params,x)
  dloss_dw2 = dloss_dyp * dmodel_dw2 (params,x)
  dloss_dw1 = dloss_dyp * dmodel_dw1 (params,x)
  dloss_dw0 = dloss_dyp * dmodel_dw0 (params,x)
  dloss_db = dloss_dyp * dmodel_db(params)
  return torch.stack([dloss_dw4.sum(),dloss_dw3.sum(),dloss_dw2.sum(),dloss_dw1.sum(),dloss_dw0.sum(), dloss_db.sum()])
def training_loop(n_epochs, learning_rate, params, x, y):
  for epoch in range(1, n_epochs+1):
    y_p = model(params,x)
    loss = loss_fn(y_p, y)
    grad = grad_fn(x,y,y_p,params)
    params = params - learning_rate * grad
    if (epoch % 500 == 0):
      print('Epoch %d, Training Loss %f' % (epoch, float(loss)))
  return params

In [ ]:
#convert input/output to tensots
x = torch.tensor(x_df.values, dtype = torch.float64)
y = torch.tensor(housing_df['price'].values, dtype = torch.float64)

In [ ]:
new_x = torch.nn.functional.normalize(x,dim=0)

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0])
params = training_loop(5000, 0.1, params, new_x, y)

In [ ]:
#batching
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

In [ ]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

In [ ]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [ ]:
import torch.nn as nn

linear_model = nn.Linear(1, 1) # <1>
linear_model(t_un_val)

In [ ]:
linear_model.weight


In [ ]:
linear_model.bias

In [ ]:
x = torch.ones(1)
linear_model(x)

In [ ]:
x = torch.ones(10, 1)
linear_model(x)

In [ ]:
linear_model = nn.Linear(1, 1) # <1>
optimizer = optim.SGD(
    linear_model.parameters(), # <2>
    lr=1e-2)

In [ ]:
linear_model.parameters()

In [ ]:
training = 0.8
validation = 0.2

def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 25 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item()*training:.4f},"
                  f" Validation loss {loss_val.item()*validation:.4f}")

In [ ]:

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

# rebuilfing linear model with nn module

linear_model = nn.Linear(1, 1) # <1>
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = loss_fn,
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

In [ ]:
linear_model.parameters()

In [ ]:
list(linear_model.parameters())

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

linear_model = nn.Linear(1, 1) # <1>
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = linear_model,
    loss_fn = loss_fn,
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print()
print(linear_model.weight)
print(linear_model.bias)

In [ ]:
#building our neuaral network
seq_model = nn.Sequential(
            nn.Linear(1, 8), # <1>
            nn.Tanh(),
            nn.Linear(8, 1)) # <2>

In [ ]:
train_size = 0.8
valid_size = 0.2

def training_loop(n_epochs, learning_rate, params, x, y):
  for epoch in range(1, n_epochs+1):
    y_p = model(params,x)
    loss = loss_fn(y_p, y)
    grad = grad_fn(x,y,y_p,params)
    params = params - learning_rate * grad

    if (epoch % 25 == 0):
      print('Epoch %d, Training Loss %f, Validation loss %f' % (epoch, float(loss*train_size), float(loss * valid_size)))
  return params

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0])
params = training_loop(200, 0.1, params, new_x, y)



In [ ]:
[param.shape for param in seq_model.parameters()]

In [ ]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

In [ ]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(1, 8)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(8, 1))
]))

seq_model

In [ ]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

In [ ]:
seq_model.output_linear.bias

In [ ]:
# Hidden layer 2
seq_model2 = nn.Sequential(
             nn.Linear(1, 8), # <input layer>
             nn.Tanh(),
             nn.Linear(8, 6), # <layer 1>
             nn.Tanh(),
             nn.Linear(6, 4), # <layer 2>
             nn.Tanh(),
             nn.Linear(4, 1), # <output layer>
             nn.Tanh())

seq_model


In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 25 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = seq_model2,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)
    
print('output', seq_model(t_un_val))
print('answer', t_c_val)
print('hidden', seq_model.hidden_linear.weight.grad)

In [ ]:
from matplotlib import pyplot as plt

t_range = torch.arange(20., 90.).unsqueeze(1)

fig = plt.figure(dpi=600)
plt.xlabel("Fahrenheit")
plt.ylabel("Celsius")
plt.plot(t_u.numpy(), t_c.numpy(), 'o')
plt.plot(t_range.numpy(), seq_model(0.1 * t_range).detach().numpy(), 'c-')
plt.plot(t_u.numpy(), seq_model(0.1 * t_u).detach().numpy(), 'kx')

In [ ]:
# Problem 2
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

In [ ]:
from torchvision import datasets
from torchvision import transforms

data_path = '/content/drive/MyDrive/Colab Notebooks/ECGR_4106_Homework/Homework1/Text_Files/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True) # <1>
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True) # <2>
   

In [ ]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [ ]:
#10 subclasses
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

fig = plt.figure(figsize=(8,3))
num_classes = 10
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    ax.set_title(class_names[i])
    img = next(img for img, label in cifar10 if label == i)
    plt.imshow(img)
plt.show()

In [ ]:
type(cifar10).__mro__

In [ ]:
len(cifar10)

In [ ]:
#Python Imaging Library, the PIL package
img, label = cifar10[99]
img, label, class_names[label]

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
from torchvision import transforms
dir(transforms)

In [ ]:
#convert image to pytorch tensor
from torchvision import transforms

to_tensor = transforms.ToTensor()
img_t = to_tensor(img)
img_t.shape

In [ ]:
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                          transform=transforms.ToTensor())

In [ ]:
img_t.shape, img_t.dtype

In [ ]:
img_t.min(), img_t.max()


In [ ]:
#verify that we’re getting the same image out:
plt.imshow(img_t.permute(1, 2, 0))  # <1>
plt.show()

In [ ]:
#normalize data
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
imgs.shape

In [ ]:
imgs.view(3, -1).mean(dim=1)  # <1>

In [ ]:
imgs.view(3, -1).std(dim=1)


In [ ]:
transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))

In [ ]:
transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
img_t, _ = transformed_cifar10[99]

plt.imshow(img_t.permute(1, 2, 0))
plt.show()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [ ]:
# download cifar10
from torchvision import datasets, transforms
data_path = '/content/drive/MyDrive/Colab Notebooks/ECGR_4106_Homework/Homework1/Text_Files/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
        ]))

In [ ]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [ ]:
#pick 512 hidden features
import torch.nn as nn

n_out = 2

model = nn.Sequential(
            nn.Linear(
                3072,  # <1>
                512,   # <2>
            ),
            nn.Tanh(),
            nn.Linear(
                512,   # <2>
                n_out, # <3>
            )
        )

In [ ]:
#specify the dimension along which the softmax function is applied.
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

In [ ]:
softmax = nn.Softmax(dim=1)

log_softmax = nn.LogSoftmax(dim=1)

x = torch.tensor([[0.0, 104.0]])

softmax(x)

In [ ]:
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.Softmax(dim=1))

In [ ]:
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.LogSoftmax(dim=1))

In [ ]:
loss = nn.NLLLoss()

In [ ]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10 
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [ ]:
import torch.nn as nn

n_out = 2

model = nn.Sequential(
            nn.Linear(
                3072,  # <1>
                512,   # <2>
            ),
            nn.Tanh(),
            nn.Linear(
                512,   # <2>
                n_out, # <3>
            )
        )

In [ ]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0])

softmax(x)

In [ ]:
softmax(x).sum()

In [ ]:
softmax = nn.Softmax(dim=1)

x = torch.tensor([[1.0, 2.0, 3.0],
                  [1.0, 2.0, 3.0]])

softmax(x)

In [ ]:
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.Softmax(dim=1))

In [ ]:
#epoch samples for single hidden layer (validation and training)

import torch
import torch.nn as nn
import torch.optim as optim

model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.LogSoftmax(dim=1))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 300

for epoch in range(n_epochs):
    for img, label in cifar2:
        out = model(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label]))
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch % 25 == 0):
      print('Epoch %d, Loss %f' % (epoch, float(loss)))


Epoch 0, Loss 4.854689


In [ ]:
#epoch samples for single hidden layer

def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 25 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)
    
print('output', seq_model(t_un_val))
print('answer', t_c_val)
print('hidden', seq_model.hidden_linear.weight.grad)

In [ ]:
#increaing the model complexity

model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2),
            nn.LogSoftmax(dim=1))

In [ ]:


model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2))

loss_fn = nn.CrossEntropyLoss()

In [ ]:
#epoch samples for multiple hidden layers (validation and training loss)

def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 25 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [ ]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val, 
    t_c_train = t_c_train,
    t_c_val = t_c_val)
    
print('output', seq_model(t_un_val))
print('answer', t_c_val)
print('hidden', seq_model.hidden_linear.weight.grad)

In [ ]:
#epoch samples for multiple hidden layers
learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 200

for epoch in range(n_epochs):
    for img, label in cifar2:
        out = model(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label]))
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch % 25 == 0):
      print('Epoch %d,  Loss %f' % (epoch, float(loss)))


Epoch 0,  Loss 360.389191
